In [136]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

#show all columns of a dataset
pd.pandas.set_option('display.max_columns',None)

#TRAIN/TEST SPLIT
from sklearn.model_selection import train_test_split


## for feature selection
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [145]:
df = pd.read_csv('SaraDataset1.csv')
df

,Type,Location,Bath,Purpose,Bedrooms,Added,Precinct,Main Features,Total Area,Total Price
0,1,"Bahria Town, Karachi",3,For Sale,3,3 hours ago,Precinct 12,Main FeaturesBuilt in year: 2022Parking Spaces...,125,14500000
1,2,"Bahria Town, Karachi",3,For Sale,2,4 hours ago,Precinct 19,Main FeaturesBuilt in year: 2022Parking Spaces...,106,7500000
2,1,"Bahria Town, Karachi",6,For Sale,5,5 hours ago,Bahria Town,Main FeaturesBuilt in year: 2024Parking Spaces...,500,37500000
3,1,"Bahria Town, Karachi",3,For Sale,3,1 day ago,Bahria Town,Main FeaturesParking Spaces: 1FlooringElectric...,125,14000000
4,1,"Bahria Town, Karachi",3,For Sale,3,1 day ago,Precinct 10-A,Main FeaturesBuilt in year: 2022Parking Spaces...,200,17500000
...,...,...,...,...,...,...,...,...,...,...
2329,1,"Bahria Town, Karachi",3,For Sale,3,3 weeks ago,Precinct 11-A,Main FeaturesParking Spaces: 1FlooringElectric...,152,19000000
2330,1,"Bahria Town, Karachi",3,For Sale,3,3 weeks ago,Precinct 11-B,Main FeaturesParking Spaces: 1FlooringElectric...,152,17500000
2331,1,"Bahria Town, Karachi",4,For Sale,4,3 weeks ago,Sports City,Main FeaturesParking Spaces: 1FlooringElectric...,350,27000000
2332,1,"Bahria Town, Karachi",5,For Sale,5,3 weeks ago,Bahria Paradise,Main FeaturesParking Spaces: 1FlooringElectric...,500,51000000


In [146]:
df.drop(columns =["Purpose","Added", "Main Features"], inplace =True, axis=1)

In [147]:
df.to_csv("Modelling.csv", index=False)

In [148]:
df = pd.read_csv('Modelling.csv')
df

,Type,Location,Bath,Bedrooms,Precinct,Total Area,Total Price
0,1,"Bahria Town, Karachi",3,3,Precinct 12,125,14500000
1,2,"Bahria Town, Karachi",3,2,Precinct 19,106,7500000
2,1,"Bahria Town, Karachi",6,5,Bahria Town,500,37500000
3,1,"Bahria Town, Karachi",3,3,Bahria Town,125,14000000
4,1,"Bahria Town, Karachi",3,3,Precinct 10-A,200,17500000
...,...,...,...,...,...,...,...
2329,1,"Bahria Town, Karachi",3,3,Precinct 11-A,152,19000000
2330,1,"Bahria Town, Karachi",3,3,Precinct 11-B,152,17500000
2331,1,"Bahria Town, Karachi",4,4,Sports City,350,27000000
2332,1,"Bahria Town, Karachi",5,5,Bahria Paradise,500,51000000


In [149]:
#df["Total Area"] = pd.to_numeric(df["Total Area"])

In [150]:
#df.drop(columns =["Purpose","Added", "Main Features"], inplace =False, axis=1)

In [151]:
categorical_feature=[feature for feature in df.columns if df[feature].dtype == 'O']
df[categorical_feature].head()

,Location,Precinct
0,"Bahria Town, Karachi",Precinct 12
1,"Bahria Town, Karachi",Precinct 19
2,"Bahria Town, Karachi",Bahria Town
3,"Bahria Town, Karachi",Bahria Town
4,"Bahria Town, Karachi",Precinct 10-A


In [152]:
df

,Type,Location,Bath,Bedrooms,Precinct,Total Area,Total Price
0,1,"Bahria Town, Karachi",3,3,Precinct 12,125,14500000
1,2,"Bahria Town, Karachi",3,2,Precinct 19,106,7500000
2,1,"Bahria Town, Karachi",6,5,Bahria Town,500,37500000
3,1,"Bahria Town, Karachi",3,3,Bahria Town,125,14000000
4,1,"Bahria Town, Karachi",3,3,Precinct 10-A,200,17500000
...,...,...,...,...,...,...,...
2329,1,"Bahria Town, Karachi",3,3,Precinct 11-A,152,19000000
2330,1,"Bahria Town, Karachi",3,3,Precinct 11-B,152,17500000
2331,1,"Bahria Town, Karachi",4,4,Sports City,350,27000000
2332,1,"Bahria Town, Karachi",5,5,Bahria Paradise,500,51000000


In [153]:
categorical_feature

['Location', 'Precinct']

In [154]:
for feature in categorical_feature:
    temp=df.groupby(feature)['Total Price'].count()/len(df)
    temp_df=temp[temp>0.01].index
    df[feature]=np.where(df[feature].isin(temp_df),df[feature],'Rare_variables')
    

In [155]:
df.head(50)

,Type,Location,Bath,Bedrooms,Precinct,Total Area,Total Price
0,1,"Bahria Town, Karachi",3,3,Precinct 12,125,14500000
1,2,"Bahria Town, Karachi",3,2,Precinct 19,106,7500000
2,1,"Bahria Town, Karachi",6,5,Bahria Town,500,37500000
3,1,"Bahria Town, Karachi",3,3,Bahria Town,125,14000000
4,1,"Bahria Town, Karachi",3,3,Precinct 10-A,200,17500000
5,1,"Bahria Town, Karachi",3,3,Precinct 31,235,15300000
6,1,"Bahria Town, Karachi",3,3,Precinct 2,152,20000000
7,1,"Bahria Town, Karachi",3,3,Precinct 12,125,15500000
8,1,"Bahria Town, Karachi",4,4,Sports City,350,26000000
9,1,"Bahria Town, Karachi",5,5,Precinct 1,272,42500000


In [156]:
for feature in categorical_feature:
    labels_ordered=df.groupby([feature])['Total Price'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df[feature]=df[feature].map(labels_ordered)

In [157]:
df.head(100)

,Type,Location,Bath,Bedrooms,Precinct,Total Area,Total Price
0,1,0,3,3,2,125,14500000
1,2,0,3,2,0,106,7500000
2,1,0,6,5,1,500,37500000
3,1,0,3,3,1,125,14000000
4,1,0,3,3,7,200,17500000
...,...,...,...,...,...,...,...
95,1,0,3,3,5,200,20500000
96,1,0,3,3,5,200,19500000
97,2,0,2,2,1,96,9288000
98,2,0,2,2,1,96,9460000


## 6. FEATURE SCALING

In [158]:
scaling_feature=[feature for feature in df.columns if feature not in ['Total Price'] ]
len(scaling_feature)

6

In [159]:
scaling_feature

['Type', 'Location', 'Bath', 'Bedrooms', 'Precinct', 'Total Area']

In [160]:
df.head()

,Type,Location,Bath,Bedrooms,Precinct,Total Area,Total Price
0,1,0,3,3,2,125,14500000
1,2,0,3,2,0,106,7500000
2,1,0,6,5,1,500,37500000
3,1,0,3,3,1,125,14000000
4,1,0,3,3,7,200,17500000


In [161]:
feature_scale=[feature for feature in df.columns if feature not in ['Total Price']]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(df[feature_scale])

MinMaxScaler()

In [162]:
scaler.transform(df[feature_scale])

array([[0.        , 0.        , 0.28571429, 0.33333333, 0.125     ,
        0.04677173],
       [1.        , 0.        , 0.28571429, 0.16666667, 0.        ,
        0.03711235],
       [0.        , 0.        , 0.71428571, 0.66666667, 0.0625    ,
        0.23741739],
       ...,
       [0.        , 0.        , 0.42857143, 0.5       , 0.8125    ,
        0.16115913],
       [0.        , 0.        , 0.57142857, 0.66666667, 0.9375    ,
        0.23741739],
       [0.        , 0.        , 0.28571429, 0.33333333, 0.375     ,
        0.10269446]])

In [163]:
# transform the train and test set, and add on the Id and SalePrice variables
data = pd.concat([df[['Total Price']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(df[feature_scale]), columns=feature_scale)],
                    axis=1)

In [164]:
data.head()

,Total Price,Type,Location,Bath,Bedrooms,Precinct,Total Area
0,14500000,0.0,0.0,0.285714,0.333333,0.1250,0.046772
1,7500000,1.0,0.0,0.285714,0.166667,0.0000,0.037112
2,37500000,0.0,0.0,0.714286,0.666667,0.0625,0.237417
3,14000000,0.0,0.0,0.285714,0.333333,0.0625,0.046772
4,17500000,0.0,0.0,0.285714,0.333333,0.4375,0.084901


In [165]:
data.to_csv('X_train.csv',index=False)

In [166]:
dataset = pd.read_csv('X_train.csv')

In [167]:
dataset.head()

,Total Price,Type,Location,Bath,Bedrooms,Precinct,Total Area
0,14500000,0.0,0.0,0.285714,0.333333,0.1250,0.046772
1,7500000,1.0,0.0,0.285714,0.166667,0.0000,0.037112
2,37500000,0.0,0.0,0.714286,0.666667,0.0625,0.237417
3,14000000,0.0,0.0,0.285714,0.333333,0.0625,0.046772
4,17500000,0.0,0.0,0.285714,0.333333,0.4375,0.084901


In [168]:
dataset.shape

(2334, 7)

## TRAIN & TEST SPLIT

In [169]:
## Capture the dependent feature and save it as y
y = dataset[["Total Price"]]

## drop dependent feature from dataset and save it as X
X = dataset.drop(['Total Price'],axis=1)

In [170]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_train:

      Type  Location      Bath  Bedrooms  Precinct  Total Area
1577   0.0       0.0  0.428571  0.500000    0.7500    0.161159
2089   1.0       0.0  0.000000  0.000000    0.0625    0.010168
1098   0.0       0.0  0.285714  0.333333    0.3125    0.084901
207    0.0       0.0  0.428571  0.500000    0.8125    0.161159
1217   1.0       0.0  0.285714  0.333333    0.5000    0.068124
(1867, 6)

X_test:

     Type  Location      Bath  Bedrooms  Precinct  Total Area
830   0.0       0.0  0.714286  0.833333    0.5000    0.237417
125   0.0       0.0  0.285714  0.333333    0.4375    0.084901
490   0.0       0.0  0.571429  0.666667    0.9375    0.237417
802   0.0       0.0  0.285714  0.333333    0.3125    0.084901
836   1.0       0.0  0.142857  0.166667    0.0625    0.037112
(467, 6)


### 1.1 Random Forest Regression Model

In [171]:
# Random Forest Regression imported from library and used for modelling 

rf = RandomForestRegressor(n_estimators=300,random_state=0)
rf.fit(X_train,y_train.values.ravel())
rf.score(X_test,y_test)
rf_pred =rf.predict(X_test)
rf_score=rf.score(X_test,y_test)
expl_rf = explained_variance_score(rf_pred,y_test)

### 1.2 Decision Tree Model

In [172]:
dtr = DecisionTreeRegressor(random_state=0)
dtr.fit(X_train,y_train)
dtr.score(X_test,y_test)
pred_tr = dtr.predict(X_test)
decision_score=dtr.score(X_test,y_test)
expl_tr = explained_variance_score(pred_tr,y_test)

### 1.3 Multiple Linear Regression

In [173]:
mlr = LinearRegression()
mlr.fit(X_train,y_train)
mlr_score = mlr.score(X_test,y_test)
pred_mlr = mlr.predict(X_test)
expl_mlr = explained_variance_score(pred_mlr,y_test)

In [174]:
print("Multiple Linear Regression Model Score is ",round(mlr.score(X_test,y_test)*100))
print("Decision tree  Regression Model Score is ",round(dtr.score(X_test,y_test)*100))
print("Random Forest Regression Model Score is ",round(rf.score(X_test,y_test)*100))

#Let's have a tabular pandas data frame, for a clear comparison

models_score =pd.DataFrame({'Model':['Multiple Linear Regression','Decision Tree','Random forest Regression'],
                            'Score':[mlr_score,decision_score,rf_score],
                            'Explained Variance Score':[expl_mlr,expl_tr,expl_rf]
                           })
models_score.sort_values(by='Score',ascending=False)

Multiple Linear Regression Model Score is  78
Decision tree  Regression Model Score is  91
Random Forest Regression Model Score is  92


,Model,Score,Explained Variance Score
2,Random forest Regression,0.917773,0.919540
1,Decision Tree,0.909560,0.912106
0,Multiple Linear Regression,0.781865,0.794950
